In [1]:
import os
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
print(os.environ.get('IEX_SANDBOX_TOKEN'))
print(os.environ.get('IEX_TOKEN'))

Tsk_2f36139a7da4433bbc4882148386ff32
sk_adc1a33884d64226b0ad7b2e4a5f213e


In [3]:
base_url = 'https://cloud.iexapis.com/v1'
sandbox_url = 'https://sandbox.iexapis.com/stable'

token = os.environ.get('IEX_TOKEN')
sandbox_token = os.environ.get('IEX_SANDBOX_TOKEN')
params = {'token': token}
sandbox_params = {'token': sandbox_token}


In [52]:
endpoint = '/stock/aapl/chart/3m'
# endpoint = '/stock/aapl/balance-sheet'
# endpoint = '/stock/aapl/financials'
# endpoint = '/time-series/reported_financials/AAPL/10-Q?last=2'
# endpoint = '/time-series'
# endpoint = '/stock/aapl/financials?period=quarter'

# endpoint = '/stock/aapl/price'
# endpoint = '/stock/aapl/income/10-Q?last=2'
# resp = requests.get(base_url+endpoint, params = params)
resp = requests.get(sandbox_url+endpoint, params = sandbox_params)
resp.raise_for_status()
resp = resp.json()

In [58]:
# resp

In [16]:
# If you plan to use this same endpoint a few times, it makes sense to create a function to avoid repeating code. Here is an example:

## Method 1: Corporate Profitability -- Profit Analysis

In [30]:
def profitability(_symbol, _url='sandbox', _last=12):
    # Here we will calculate two indicators
    # 1. ROE = net income / (asset - liability) = Net Income from Continuing Operation net minority interest (from income statement) /Stockholders' Equity (from balance sheet)
    # 2. gross profit
    
    # define parameters
    base_url = 'https://cloud.iexapis.com/v1'
    sandbox_url = 'https://sandbox.iexapis.com/stable'

    base_token = os.environ.get('IEX_TOKEN')
    sandbox_token = os.environ.get('IEX_SANDBOX_TOKEN')
    base_params = {'token': base_token}
    sandbox_params = {'token': sandbox_token}
    
    ticker_lower = f'{_symbol}'.lower() #IEX only accepts lower case ticker
    ticker_upper = f'{_symbol}'.upper() #Use upper case ticker for plotting
    
    incomeStat_url = 'stock/aapl/income'
    balanceStat_url = '/stock/aapl/balance-sheet'
    
    # _last defaul is 12; min is 1 (quarters)
    if _url == 'sandbox':
        incomeStat_endpoint = f'{sandbox_url}/{incomeStat_url}/?last={_last}'
        balanceStat_endpoint = f'{sandbox_url}/{balanceStat_url}/?last={_last}'
        incomeStat_resp = requests.get(incomeStat_endpoint, params=sandbox_params)
        balanceStat_resp = requests.get(balanceStat_endpoint, params=sandbox_params)
    else: # need IEX subscription in order to support this url
        incomeStat_endpoint = f'{base_url}/{incomeStat_url}/?last={_last}'
        balanceStat_endpoint = f'{base_url}/{balanceStat_url}/?last={_last}'
        incomeStat_resp = requests.get(incomeStat_endpoint, params=base_params) 
        balanceStat_resp = requests.get(balanceStat_endpoint, params=base_params)
    incomeStat_resp.raise_for_status()
    incomeStat_resp = incomeStat_resp.json()
    if _last == 12 and len(incomeStat_resp['income']) < 12:
        print(f'Hey! {ticker_upper} is a relatively new IPO.\nThe company has published less than 3 years worth of financial data for us to perform a meaningful analysis.')
        return None
    balanceStat_resp.raise_for_status()
    balanceStat_resp = balanceStat_resp.json()
    
    return incomeStat_resp
    

In [31]:
profitability('nio', 'sandbox',2)

{'symbol': 'AAPL',
 'income': [{'costOfRevenue': 52781878745,
   'currency': 'USD',
   'ebit': 28092323582,
   'filingType': '10-Q',
   'fiscalDate': '2021-03-26',
   'fiscalQuarter': 2,
   'fiscalYear': 2031,
   'grossProfit': 39041746447,
   'incomeTax': 4403263573,
   'interestIncome': 696546328,
   'minorityInterest': 0,
   'netIncome': 24342818360,
   'netIncomeBasic': 24571718942,
   'operatingExpense': 63889503802,
   'operatingIncome': 28189524065,
   'otherIncomeExpenseNet': 0,
   'pretaxIncome': 29405766281,
   'reportDate': '2021-04-29',
   'researchAndDevelopment': 5338598255,
   'sellingGeneralAndAdmin': 5485123228,
   'symbol': 'AAPL',
   'totalRevenue': 92804751009,
   'id': 'NECIMO',
   'key': 'ALPA',
   'subkey': 'quarterly',
   'updated': 1625122979887},
  {'costOfRevenue': 69469727207,
   'currency': 'USD',
   'ebit': 35209618931,
   'filingType': '10-Q',
   'fiscalDate': '2020-12-24',
   'fiscalQuarter': 1,
   'fiscalYear': 2112,
   'grossProfit': 45369781809,
   'i

In [ ]:
# _last defaul is 12; min is 1 (quarters)
endpoint = f'{sandbox_url}/{incomeStat_endpoint}'
resp = requests.get(endpoint, params=sandbox_params)
resp = requests.get(endpoint, params=base_params) # need IEX subscription in order to support this url
resp.raise_for_status()
resp = resp.json()

## Method 4: Corporate Growth -- Revenue Analysis

In [28]:
def growth(_symbol, _url='sandbox', _last=12):
    
    # define parameters
    base_url = 'https://cloud.iexapis.com/v1'
    sandbox_url = 'https://sandbox.iexapis.com/stable'

    base_token = os.environ.get('IEX_TOKEN')
    sandbox_token = os.environ.get('IEX_SANDBOX_TOKEN')
    base_params = {'token': base_token}
    sandbox_params = {'token': sandbox_token}
    
    ticker_lower = f'{_symbol}'.lower() #IEX only accepts lower case ticker
    ticker_upper = f'{_symbol}'.upper() #Use upper case ticker for plotting
    
    # _last defaul is 12; min is 1 (quarters)
    if _url == 'sandbox':
        endpoint = f'{sandbox_url}/stock/{ticker_lower}/income/10-Q/?last={_last}'
        resp = requests.get(endpoint, params=sandbox_params)
    else:
        endpoint = f'{base_url}/stock/{ticker_lower}/income/10-Q/?last={_last}'
        resp = requests.get(endpoint, params=base_params) # need IEX subscription in order to support this url
    resp.raise_for_status()
    resp = resp.json()
    
    if _last == 12 and len(resp['income']) < 12:
        print(f'Hey! {ticker_upper} is a relatively new IPO.\nThe company has published less than 3 years worth of financial data for us to perform a meaningful analysis.')
        return None
    
    rev = []
    rev_qrt = {}
    
    # extract date / quarter_number / quarterly_revenue
    for num in range(0,len(resp['income'])):
        for key in ('fiscalDate','fiscalQuarter','totalRevenue'):
            rev_qrt[key] = resp['income'][num].setdefault(key)
        rev.append(rev_qrt)
        rev_qrt = {}
    
    # add TTM_Revenue
    # the last 3 TTM Revenue would not be available
    for i in range(0,len(rev)):
        if (i+3) < len(rev):
            rev[i]['TTM_Revenue'] = rev[i]['totalRevenue'] + rev[i+1]['totalRevenue'] + rev[i+2]['totalRevenue'] + rev[i+3]['totalRevenue']
   
    # Calculate YoY quarterly & yearly revenue growth rate
    q=0
    y=0
    while q < 8: # YOY quarterly growth is calculated every 4 quarters, so 12 quarters of data can only derive 8 data points
        rev[q]['Quarterly_Revenue_Growth'] = round((rev[q]['totalRevenue']/rev[q+4]['totalRevenue']-1)*100,2)
        q += 1
    while y < 5: # YOY same period growth is calculated every 4 data points, so 12 quarters of data can only derive 5 data points   
        rev[y]['TTM_Revenue_Growth'] = round((rev[y]['TTM_Revenue']/rev[y+4]['TTM_Revenue']-1)*100,2)
        y += 1
    
    # Convert to pandas DF
    rev_df = pd.DataFrame(rev)  
    # rename totalRevenue to Quarterly_Revenue
    rev_df = rev_df.rename(columns={'totalRevenue': 'Quarterly_Revenue'})
    
    # remove df rows where quarterly_revenue_growth is NaN
    rev_df = rev_df.loc[0:7]
    # use datetime format and sort df by date
    rev_df['fiscalDate'] = pd.to_datetime(rev_df['fiscalDate'], format='%Y/%m/%d')
    rev_df = rev_df.sort_values('fiscalDate')
    # transform date to year-month format
    rev_df["date_yrmo"] = rev_df['fiscalDate'].dt.strftime('%Y-%m')
    
    # calculate monthly average stock price 
    if _url == 'sandbox':
        endpoint = f'{sandbox_url}/stock/{ticker_lower}/chart/39m'
        resp = requests.get(endpoint, params=sandbox_params)
    else:
        endpoint = f'{base_url}/stock/{ticker_lower}/chart/39m'
        resp = requests.get(endpoint, params=base_params) # need IEX subscription in order to support this url
    
    resp.raise_for_status()
    df = pd.DataFrame(resp.json())
    df = df[["symbol","date","fClose","fVolume"]]
    # transform date to year-month format
    df["date"] = pd.to_datetime(df['date'], format='%Y/%m/%d')
    df["date_yrmo"] = df['date'].dt.strftime('%Y-%m')
    # average close price by month
    df1 = df.groupby('date_yrmo')['fClose'].mean().reset_index(name='avg_monthly_price')
    df1['avg_monthly_price'] = round(df1['avg_monthly_price'],2)
    df = pd.merge(df, df1, on='date_yrmo', how='left')
    # merge df
    rev_df = pd.merge(rev_df, df, on='date_yrmo', how='left')
    
    
    #---------------------------------------------------------------------------------------------------------------------------------------------
    # Plotting
    fig = plt.figure(figsize=(16,9), dpi=100)
    ax = fig.add_subplot(111)

    ln1 = ax.plot(rev_df.date_yrmo, rev_df.Quarterly_Revenue_Growth, '-', label = 'Quarterly Revenue Growth Rate', color='tab:blue', linewidth=8)
    ln2 = ax.plot(rev_df.date_yrmo, rev_df.TTM_Revenue_Growth, '-', label = 'TTM Revenue Growth Rate', color='tab:green', linewidth=8)
    ax2 = ax.twinx()
    ln3 = ax2.plot(rev_df.date_yrmo, rev_df.avg_monthly_price, 'r', label = 'Monthly Avg Stock Price', color='tab:red', linewidth=3)

    # combine three lines
    ln = ln1+ln2+ln3
    labs = [l.get_label() for l in ln]

    ax.legend(ln, labs, loc='upper right')#, ncol=3, mode = "expand")
    ax.set_title(f'{ticker_upper}: YoY Quarterly vs TTM Revenue Growth Rate and Stock Price', fontsize=16)
    ax.grid(alpha=.2)
    ax.set_xlabel('Fiscal Year', fontsize=16)
    ax.set_ylabel('Revenue Growth Rate (%)', fontsize=16)
    ax2.set_ylabel('Monthly Average Stock Price ($)', fontsize=16)
    fig.tight_layout()
    plt.show()
    #----------------------------------------------------------------------------------------------------------------------------------------------
   
    # return simple revenue history
    rev = pd.DataFrame(rev)
    rev = rev.sort_values('fiscalDate')
    return rev
#     return None
    

In [29]:
revenue_growth_trend = growth('nio', 'sandbox', 12)
revenue_growth_trend


Hey! NIO is a relatively new IPO.
The company has published less than 3 years worth of financial data for us to perform a meaningful analysis.
